In [1]:
!pip install langchain-core==0.2.42
!pip install langchain-groq
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.11
    Uninstalling langsmith-0.3.11:
      Successfully uninstalled langsmith-0.3.11
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires langchain-core<1.0.0,>=0.3.35, but you have langchain-core 0.2.42 which is incompatible.
langchain-text-splitters 0.3.6 requ

In [2]:
from getpass import getpass
api_key = getpass('Enter your API key: ')

Enter your API key: ··········


In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=api_key,
    model_name="llama-3.1-8b-instant"
)

Extracting Job post content

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://mycareer.hsbc.com/en_GB/external/PipelineDetail/Business-Analyst/257236")
page_data = loader.load().pop().page_content
print(page_data)

from langchain_core.prompts import PromptTemplate

prompt_extract= PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
    ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Provide the output in JSON format.
    """
)

chain_extract = prompt_extract | llm
result = chain_extract.invoke(input={'page_data': page_data})
print(result.content)

from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(result.content)
print(json_res)



















        Error | HSBC Group    












 









                        Skip to content                    




                                    HSBC Group
                                




            Menu        






                        Careers at HSBC                    



                        Early careers                    



                        Where we hire                    



                        Job alert                    



                        Contacts                    





                        My Profile                    






                                Current language                            

English - UKEspañol - ESFrançais - CanadaFrançais - France中文 - 简体














                                Search Jobs
                            






                                    Error
                                








                                        Explore exciting career oppo

Extracting Resume content

In [5]:
from langchain_community.document_loaders import PyPDFLoader
!pip install pypdf

pdfloader = PyPDFLoader("sample resume.docx.pdf")
doc_data = pdfloader.load()
resume_content = "\n".join([doc.page_content for doc in doc_data])

from langchain_core.prompts import PromptTemplate

prompt_extract_resume= PromptTemplate.from_template(
    """
    Extract the following details from the given resume document contents
    - Full Name
    - Contact Information (Email, Phone, LinkedIn, Github, Other socials)
    - Summary
    - Work Experience (Company, Role, Duration, Responsibilities)
    - Education (Institution, Degree, Year, Coursework)
    - Skills
    - Certifications (if any)
    - Projects (if mentioned)

    Resume content: {resume_content}

    Provide the output in JSON format
    NO PREAMBLE
    """
)

chain_extract_resume = prompt_extract_resume | llm
resume_result = chain_extract_resume.invoke(input={'resume_content': resume_content})
print(resume_result.content)

from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_resume_res = json_parser.parse(resume_result.content)
print(json_resume_res)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 3.4 MB/s eta 0:00:00
{
  "Full Name": "Alex Carter",
  "Contact Information": {
    "Email": "acarter@example.com",
    "Phone": "+1 555-123-4567",
    "LinkedIn": "linkedin.com/in/alex-carter",
    "Github": "github.com/alexdev",
    "Other socials": ""
  },
  "Summary": "",
  "Work Experience": [
    {
      "Company": "InnovateTech Solutions",
      "Role": "Software Engineering Intern",
      "Duration": "June 2024 - August 2024",
      "Responsibilities": [
        "Developed and optimized backend services, reducing API response time by 30%",
        "Assisted in the development of machine learning models for predictive analytics, improving data-driven decision-making.",
        "Collaborated with cross-functional teams to integrate new features into existing platforms, enhancing user experience."
      ]
    },
    {
      "Company": "CodeWave Hackathon Team",
      "Role": "Team Lead",
      "Duration": "September 2023 -

Email generation based on given job post and resume

In [6]:
prompt_email = PromptTemplate.from_template(
        """

        **Prompt:**

        You are an expert copywriter known for crafting highly effective cold emails. Your goal is to write a compelling cold email for a client applying for a job.

        ### **Instructions:**
        - Use the provided job description and the client’s resume to tailor the email.
        - Highlight the client’s most relevant skills, achievements, and experiences that align with the job requirements.
        - Keep the email concise, engaging, and persuasive.
        - Use a professional yet conversational tone.
        - Ensure the email includes:
          - A strong, attention-grabbing opening.
          - A clear value proposition demonstrating why the client is a great fit.
          - A compelling call to action encouraging the hiring manager to respond.
        - Optimize formatting for easy readability (e.g., short paragraphs, bullet points if necessary).

        ### **Provided Data:**
        1. **Job Description:** {job_description}
        2. **Client Resume:** {client_resume}

        Write a cold email that maximizes the chances of getting a response. Return only the email and no other text.



        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_res), "client_resume":str(json_resume_res) })
print(res.content)

Subject: Unlocking Your Team's Potential with Alex Carter

Dear Hiring Manager,

I came across the Software Engineer position at [Company Name] and was impressed by the opportunity to join a team that's pushing the boundaries of innovation. As a seasoned software engineer with a passion for building scalable solutions, I'm excited to apply my skills and experience to drive success.

With a strong background in software engineering, I've developed a unique blend of technical expertise and problem-solving skills. My most recent role as a Software Engineering Intern at InnovateTech Solutions has given me hands-on experience in developing and optimizing backend services, reducing API response time by 30%. I've also assisted in the development of machine learning models for predictive analytics, improving data-driven decision-making.

As a team lead at CodeWave Hackathon Team, I've led a team of five in developing innovative solutions for various hackathon challenges, securing multiple top-